A. 
Write a Python function that simulates the classical Deutsch-Josza algorithm to determine whether a given binary function is constant or balanced. The function should take as input a classical oracle representing the binary function and apply the necessary operations to determine its nature (constant or balanced). The goal is to minimize the number of function evaluations required to make the determination.

B.
Write a Python function that implements the quantum Deutsch-Josza algorithm using a quantum circuit to solve the same binary function evaluation problem. The function should encode the function into qubits, apply quantum gates to perform the computation, and measure the final state to determine whether the function is constant or balanced.

In [45]:
# A. Classical Deutsch-Josza Algorithm:
def classical_deutsch_josza(oracle):
    n = len(oracle)
    is_constant = True
    is_balanced = True

    # Check if the function is constant
    for i in range(n):
        for j in range(i+1, n):
            if oracle[i] != oracle[j]:
                is_constant = False
                break

    # Check if the function is balanced
    if not is_constant:
        for i in range(n):
            if oracle[i] != oracle[(i+1) % n]:
                is_balanced = False
                break

    if is_constant:
        print("The function is constant.")
    elif is_balanced:
        print("The function is balanced.")
    else:
        print("The function is neither constant nor balanced.")
        
    return is_constant, is_balanced

# Example usage:
oracle_constant = [0, 0, 0, 0]  # Constant function
oracle_balanced = [1, 0, 0, 1]  # Balanced function
oracle_random = [0, 1, 0, 1]    # Neither constant nor balanced


classical_deutsch_josza(oracle_constant)
classical_deutsch_josza(oracle_balanced)
classical_deutsch_josza(oracle_random)

The function is constant.
The function is neither constant nor balanced.
The function is neither constant nor balanced.


(False, False)

In [1]:
import numpy as np
from math import sqrt

In [3]:
def getTensor(matrices):
    product = matrices[0]
    for matrix in matrices[1:]:
        product = np.kron(product,matrix)  ## np.kron stands for Kronecker product, the official name for tensor product
    return product

In [4]:
def U(n, f_map):
    num_qubits = n + 1
    U = np.zeros((2**num_qubits, 2**num_qubits)) # Start with a matrix of zeroes.
    
    # Quantum state looks like IN-IN-...-IN-ANCILLA
    for input_state in range(2**num_qubits): # For each possible input
        input_string = input_state >> 1 # remove ANCILLA
        output_qubit = (input_state & 1) ^ (f_map[input_string]) # remove IN, XOR with f(IN)
        output_state = (input_string << 1) + output_qubit # the full state, with new OUT
        U[input_state, output_state] = 1 # set that part of U to 1

In [7]:
def U(n, f_map):
    num_qubits = n + 1
    U = np.zeros((2**num_qubits, 2**num_qubits)) # Start with a matrix of zeroes.
    
    # Quantum state looks like IN-IN-...-IN-ANCILLA
    for input_state in range(2**num_qubits): # For each possible input
        input_string = input_state >> 1 # remove ANCILLA
        output_qubit = (input_state & 1) ^ (f_map[input_string]) # remove IN, XOR with f(IN)
        output_state = (input_string << 1) + output_qubit # the full state, with new OUT
        U[input_state, output_state] = 1 # set that part of U to 1

    return U

In [8]:
def measure(n, state):
    measurement = np.zeros(2**n)  # Initialize measurement result for n qubits in the first register
    for index, value in enumerate(state):
        measurement[index >> 1] += value * value  ## As the ancilla qubit is discarded, probabilities of the same kind, ie 100 and 101 will be combined

    # Last step: Determine the type of function f
    # f is constant if the probability of measuring |0> is positive
    if (abs(measurement[0]) > 1e-10): 
        print("The function is constant.")
    else:
        print("The function is balanced.")

In [9]:
def Deutsch_Jozsa(n, f_map):
    num_qubits = n + 1  # Plus one qubit and the second register, can be called as ancilla qubit
    state_0 = np.array([[1],[0]])  # Standard state |0> as a column vector
    I_gate = np.array([[1,0], [0,1]])  # Identity gate
    X_gate = np.array([[0,1], [1,0]])  # NOT gate
    H_gate = np.array([[1,1], [1,-1]])/sqrt(2)  # Hadamard gate
    
    ancilla = np.dot(X_gate, state_0)  # Create state |1> assigned to the ancilla
    
    # Create the a Hadamard transformation for all qubits and the state |ψ_0> 
    listStates = []
    listGates_H = []
    for i in range(n):
        listStates.append(state_0)
        listGates_H.append(H_gate)
    listStates.append(ancilla)
    listGates_H.append(H_gate)
    psi_0 = getTensor(listStates)
    composite_H = getTensor(listGates_H)
    
    # |ψ_1> is the dot product of the Hadamard transformation and |ψ_0>  
    psi_1 = np.dot(composite_H, psi_0)

    # Apply the oracle to |ψ_1>
    psi_2 = np.dot(U(n, f_map), psi_1)

    # H on all again
    psi_3 = np.dot(composite_H, psi_2)

    measure(n, psi_3)

In [11]:
def main():
    n = [2,3,3]  # Input the number of qubits
    f_map = [[0,0,1,1],
             [1,1,1,1,1,1,1,1],
             [1,0,0,1,1,0,1,0]]  # Input the mapping functions
    for index, value in enumerate(n):
        Deutsch_Jozsa(n[index], f_map[index])  # Algorithm executed here
        
main()

The function is balanced.
The function is constant.
The function is balanced.


In [18]:
from qiskit import QuantumCircuit
# Create a circuit with a register of three qubits
circ = QuantumCircuit(3)
# H gate on qubit 0, putting this qubit in a superposition of |0> + |1>.
circ.h(0)
# A CX (CNOT) gate on control qubit 0 and target qubit 1 generating a Bell state.
circ.cx(0, 1)
# CX (CNOT) gate on control qubit 0 and target qubit 2 resulting in a GHZ state.
circ.cx(0, 2)
# Draw the circuit
circ.draw('mpl')

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.